In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('diabetes_prediction_dataset.csv')

In [4]:
df.shape

(100000, 9)

In [ ]:
from sklearn.metrics import confusion_matrix
def confusion(model):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    return cm

In [5]:
display(df.head())

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [6]:
display(df.info())
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


None

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,41.885856,0.07485,0.039420,27.320767,5.527507,138.058060,0.085000
std,22.516840,0.26315,0.194593,6.636783,1.070672,40.708136,0.278883
min,0.080000,0.00000,0.000000,10.010000,3.500000,80.000000,0.000000
25%,24.000000,0.00000,0.000000,23.630000,4.800000,100.000000,0.000000
50%,43.000000,0.00000,0.000000,27.320000,5.800000,140.000000,0.000000
75%,60.000000,0.00000,0.000000,29.580000,6.200000,159.000000,0.000000
max,80.000000,1.00000,1.000000,95.690000,9.000000,300.000000,1.000000


In [ ]:
from sklearn.model_selection import train_test_split
training_column = 'diabetes'
drop_columns = ['']
X = df.drop(columns=[training_column])
y = df[training_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Preprocessing

In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

transformer = ColumnTransformer(transformers=[
    ('num', StandardScaler(), ['age', 'hypertension', 'heart_disease', 'bmi', 'HbA1c_level', 'blood_glucose_level']),
    ('cat', OneHotEncoder(), ['gender', 'smoking_history'])
])

In [38]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline(steps=[
    ('preprocessor', transformer),
    ('classifier', RandomForestClassifier(class_weight='balanced', random_state=42))
])

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
pos = (y_train==1).sum(); neg = (y_train==0).sum()
pos_weight = neg / pos
param_grid = [
    {
        'classifier': [RandomForestClassifier(class_weight='balanced', random_state=42)],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20],
        'classifier__min_samples_split': [2, 5, 10]

    },
    {
        "classifier": [xgb.XGBClassifier(
            objective="binary:logistic",
            nthread=-1,
            tree_method="hist",
            random_state=42,
            scale_pos_weight=pos_weight  # <<< key for imbalance
        )],
        "classifier__n_estimators": [300, 600],
        "classifier__max_depth": [3, 5, 7],
        "classifier__learning_rate": [0.03, 0.1],
        "classifier__subsample": [0.7, 1.0],
        "classifier__colsample_bytree": [0.7, 1.0],
        "classifier__min_child_weight": [1, 5],
        "classifier__gamma": [0, 1, 5]
    },
]
grid_search_recall = GridSearchCV(pipeline, param_grid, cv=5, scoring='recall', n_jobs=-1)
grid_search_recall.fit(X_train, y_train)

,estimator,Pipeline(step...m_state=42))])
,param_grid,"[{'classifier': [RandomForestC...ndom_state=42)], 'classifier__max_depth': [None, 10, ...], 'classifier__min_samples_split': [2, 5, ...], 'classifier__n_estimators': [50, 100, ...]}, {'classifier': [XGBClassifier...ree=None, ...)], 'classifier__colsample_bytree': [0.7, 1.0], 'classifier__gamma': [0, 1], 'classifier__learning_rate': [0.03, 0.1], ...}]"
,scoring,'recall'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [54]:
display(grid_search_recall.best_params_)

{'classifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='aucpr', feature_types=None,
               feature_weights=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=None, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=None,
               n_jobs=None, num_parallel_tree=None, ...),
 'classifier__colsample_bytree': 1.0,
 'classifier__gamma': 0,
 'classifier__learning_rate': 0.03,
 'classifier__max_depth': 3,
 'classifier__min_child_weight': 1,
 'classifier__n_estimators': 300,
 'classifier

In [55]:
display(grid_search_recall.best_score_)

np.float64(0.9250584929358739)

In [56]:
confusion(grid_search_recall)

array([[16455,  1837],
       [  128,  1580]])

In [57]:
grid_search_recall.cv_results_['mean_test_score']

array([0.68786706, 0.68668919, 0.68654191, 0.71804996, 0.71731326,
       0.7164295 , 0.76663697, 0.76442795, 0.76457555, 0.89428823,
       0.89723211, 0.90164831, 0.89914539, 0.90091248, 0.90135376,
       0.89899855, 0.90091237, 0.90429873, 0.71363257, 0.70951048,
       0.70921528, 0.74042791, 0.73983946, 0.73792553, 0.78239089,
       0.78209569, 0.78342073, 0.92358617, 0.92078892, 0.92432233,
       0.92049436, 0.92432233, 0.9203472 , 0.92432222, 0.92049436,
       0.92285045, 0.92167257, 0.92285088, 0.92167268, 0.92299805,
       0.92123107, 0.92314521, 0.92137812, 0.92358628, 0.92093608,
       0.92388072, 0.92034709, 0.92402811, 0.92078924, 0.92373356,
       0.92064153, 0.92255611, 0.92064175, 0.92285077, 0.92123085,
       0.92314543, 0.92020014, 0.9227035 , 0.92064207, 0.92034731,
       0.92108368, 0.9215254 , 0.9213778 , 0.92049436, 0.92137802,
       0.92137791, 0.92049491, 0.91224889, 0.91578296, 0.91372143,
       0.91784406, 0.91210183, 0.91475192, 0.91372143, 0.91563